In [1]:
import numpy as np
import tensorflow as tf

In [2]:
m = 100
x = tf.constant(np.random.uniform(size =(2,m)), dtype = tf.float32)
y = tf.constant(np.random.randint(2,size =(1,m)), dtype = tf.float32)

In [3]:
w1 = tf.constant(np.random.uniform(size =(4, 2)), dtype = tf.float32)
b1 = tf.constant(np.zeros(shape = (4, 1)), dtype = tf.float32)

w2 = tf.constant(np.random.uniform(size =(1, 4)), dtype = tf.float32)
b2 = tf.constant(np.zeros(shape = (1, 1)), dtype = tf.float32)

In [4]:
class Dense():
    def __init__(self, units, input_shape, activation, weights, bias):
        self.units = units
        self.input_shape = input_shape
        self.activation = activation
        self.weights = weights
        self.bias = bias
    def __call__(self, X):
        if self.activation == 'sigmoid':
            return tf.math.sigmoid(tf.matmul(self.weights, X) + self.bias)
        if self.activation == 'tanh':
            return tf.math.tanh(tf.matmul(self.weights, X) + self.bias)
        
    def update_weights(self,dw, lr):
        self.weights = self.weights - lr * dw
        
    def update_bias(self,db, lr):
        self.bias = self.bias - lr * db
        
    
    def get_weights(self):
        return self.weights, self.bias

In [5]:
h1 = Dense(units = 4, input_shape = 2, activation = 'tanh', weights = w1, bias = b1)
o1 = Dense(units = 1, input_shape = 4, activation = 'sigmoid', weights = w2, bias = b2)
y_hat = o1(h1(x))

In [93]:
h1 = Dense(units = 4, input_shape = 2, activation = 'tanh', weights = w1, bias = b1)
o1 = Dense(units = 1, input_shape = 4, activation = 'sigmoid', weights = w2, bias = b2)

lr = 0.1
EPOCHS = 10
for i in range(EPOCHS):
    y_hat = o1(h1(x))
    loss = tf.squeeze((tf.matmul(y,tf.math.log(y_hat), transpose_b = True) + tf.matmul(1 - y,tf.math.log(1 - y_hat), transpose_b = True)) * (-1/m))

    # second layer derivative calculation
    dLda2 = - ((y / y_hat) - ((1 - y)/(1 - y_hat)))
    da2dz2 = y_hat * (1 - y_hat)
    dLdz2 = dLda2 * da2dz2
    
    dw2 = tf.matmul(dLdz2, h1(x), transpose_b = True) / m
    db2 = tf.reduce_sum(dLdz2, axis = 1, keepdims = True) / m

    # first layer
    w2_matrix = o1.get_weights()9[0]
    dLdz1 = tf.matmul(w2_matrix, dLdz2, transpose_a = True) * (1 - tf.math.pow(h1(x),2))
    
    dw1 = tf.matmul(dLdz1, x, transpose_b = True) / m
    db1 = tf.reduce_sum(dLdz1, axis = 1, keepdims = True) / m

    h1.update_weights(dw1, lr)
    h1.update_bias(db1, lr)
    o1.update_weights(dw2, lr)
    o1.update_bias(db2, lr)
    print('Epoch: {}   loss: {}'.format(i+1, loss))


Epoch: 1   loss: 0.6761887073516846
Epoch: 2   loss: 0.6744393110275269
Epoch: 3   loss: 0.6728968620300293
Epoch: 4   loss: 0.6715371012687683
Epoch: 5   loss: 0.6703384518623352
Epoch: 6   loss: 0.6692817807197571
Epoch: 7   loss: 0.668350100517273
Epoch: 8   loss: 0.6675285696983337
Epoch: 9   loss: 0.6668040156364441
Epoch: 10   loss: 0.666164755821228


In [14]:
# w2_matrix = o1.get_weights()[0]
# dLdz1 = tf.matmul(w2_matrix, dLdz2, transpose_a = True) * (1 - tf.math.pow(h1(x),2))
# dLdz1.shape
(1 - tf.math.pow(h1(x),2)).shape

TensorShape([4, 100])

In [92]:
# using keras to get same results
def create_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units = 4,activation = 'tanh',  name = 'd1', input_shape = (x.shape[0],)))
    model.add(tf.keras.layers.Dense(units = 1,activation = 'sigmoid',  name = 'o1'))
    
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1),
                loss='binary_crossentropy', metrics = ['accuracy'])
    
    return model
model = create_model()

d1 = model.get_layer('d1')
d1_weights = [tf.constant(tf.transpose(w1), dtype = tf.float32), tf.constant(tf.squeeze(b1), dtype = tf.float32)]
d1.set_weights(d1_weights)

o1 = model.get_layer('o1')
o1_weights = [tf.constant(tf.transpose(w2), dtype = tf.float32), tf.constant(tf.squeeze(b2, axis = 1), dtype = tf.float32)]
o1.set_weights(o1_weights)

xt = tf.transpose(x)
yt = tf.transpose(y)
model.fit(xt, yt, epochs = 10, batch_size = 1000)


Epoch 1/10
1/1 [==============================] - 0s 2ms/step - loss: 0.6762 - accuracy: 0.6100
Epoch 2/10
1/1 [==============================] - 0s 997us/step - loss: 0.6744 - accuracy: 0.6100
Epoch 3/10
1/1 [==============================] - 0s 993us/step - loss: 0.6729 - accuracy: 0.6100
Epoch 4/10
1/1 [==============================] - 0s 998us/step - loss: 0.6715 - accuracy: 0.6100
Epoch 5/10
1/1 [==============================] - 0s 997us/step - loss: 0.6703 - accuracy: 0.6100
Epoch 6/10
1/1 [==============================] - 0s 983us/step - loss: 0.6693 - accuracy: 0.6100
Epoch 7/10
1/1 [==============================] - 0s 2ms/step - loss: 0.6684 - accuracy: 0.6100
Epoch 8/10
1/1 [==============================] - 0s 996us/step - loss: 0.6675 - accuracy: 0.6100
Epoch 9/10
1/1 [==============================] - 0s 996us/step - loss: 0.6668 - accuracy: 0.6100
Epoch 10/10
1/1 [==============================] - 0s 988us/step - loss: 0.6662 - accuracy: 0.6100
